### End to End Data Engineering Project from Getting the data, the build Data Model, Write Python to create the Table Structure, Lastly Insert Data From Files to The Database

In [1]:
import pandas as pd
import psycopg2

In [2]:
def create_database():
    #Connect to deafult database
    conn = psycopg2.connect("host=127.0.0.1 dbname=postgres user=postgres password=********")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    #Create "Accounts" database with UTF8 Encoding
    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")
    
    #close the connection to default database
    conn.close()
    
    #connect to "Accounts" database
    conn = psycopg2.connect("host=127.0.0.1 dbname=accounts user=postgres password=********")
    #conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    return cur, conn

In [3]:
def drop_tables(cur,conn):
    for query in drop_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
def create_table(cur,conn):
    for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [5]:
AccountsCountry = pd.read_csv('Dataset/Wealth-AccountsCountry.csv')

In [6]:
AccountsCountry.head()

,Code,Long Name,Income Group,Region,Lending category,Other groups,Currency Unit,Latest population census,Latest household survey,Special Notes,...,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data,2-alpha code,WB-2 code,Table Name,Short Name
0,ALB,Republic of Albania,Upper middle income,Europe & Central Asia,IBRD,NaN,Albanian lek,2020 (expected),"Demographic and Health Survey, 2017/18",NaN,...,Living Standards Measurement Study Survey (LSM...,Yes,2012,2013.0,2018.0,2006.0,AL,AL,Albania,Albania
1,ARG,Argentine Republic,Upper middle income,Latin America & Caribbean,IBRD,NaN,Argentine peso,2020 (expected),"Multiple Indicator Cluster Survey, 2019/20",NaN,...,"Integrated household survey (IHS), 2016",Yes,2008,2002.0,2018.0,2011.0,AR,AR,Argentina,Argentina
2,ARM,Republic of Armenia,Upper middle income,Europe & Central Asia,IBRD,NaN,Armenian dram,2020 (expected),"Demographic and Health Survey, 2015/16",NaN,...,"Integrated household survey (IHS), 2016",Yes,2014,NaN,2018.0,2012.0,AM,AM,Armenia,Armenia
3,AUS,Commonwealth of Australia,High income,East Asia & Pacific,NaN,NaN,Australian dollar,2016,NaN,Fiscal year end: June 30; reporting period for...,...,"Expenditure survey/budget survey (ES/BS), 2010",Yes,2015-2016,2013.0,2018.0,2013.0,AU,AU,Australia,Australia
4,AUT,Republic of Austria,High income,Europe & Central Asia,NaN,Euro area,Euro,2011. Population figures compiled from adminis...,NaN,A simple multiplier is used to convert the nat...,...,"Income survey (IS), 2015",Yes,2010,2014.0,2018.0,2010.0,AT,AT,Austria,Austria


In [7]:
AccountsCountry_clean = AccountsCountry[['Code','Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [8]:
AccountsCountry_clean.head()

,Code,Short Name,Table Name,Long Name,Currency Unit
0,ALB,Albania,Albania,Republic of Albania,Albanian lek
1,ARG,Argentina,Argentina,Argentine Republic,Argentine peso
2,ARM,Armenia,Armenia,Republic of Armenia,Armenian dram
3,AUS,Australia,Australia,Commonwealth of Australia,Australian dollar
4,AUT,Austria,Austria,Republic of Austria,Euro


In [9]:
AccountsData = pd.read_csv('Dataset/Wealth-AccountData.csv')

In [10]:
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],...,2009 [YR2009],2010 [YR2010],2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018]
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,4.34E+10,3.71E+10,3.88E+10,4.22E+10,4.36E+10,...,6.610000e+10,6.810000e+10,6.850000e+10,7.080000e+10,7.160000e+10,7.250000e+10,7.370000e+10,7.57E+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,13708.99,11794.1,12399.48,13573.26,14118.13,...,2.258200e+04,2.336286e+04,2.359058e+04,2.439397e+04,2.473162e+04,2.509002e+04,2.559371e+04,26326.26,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3355.549,2868.997,3013.391,3259.893,3424.007,...,2.825336e+03,2.889150e+03,2.848523e+03,2.930234e+03,2.954273e+03,2.957420e+03,2.987545e+03,3072.504,3178.417,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,6353.737,5466.692,5901.072,6492.73,6872.019,...,1.525319e+04,1.578992e+04,1.579854e+04,1.647360e+04,1.681013e+04,1.699138e+04,1.730701e+04,17817.1,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4737.857,4059.321,4191.445,4536.972,4695.071,...,3.529144e+03,3.613606e+03,3.603694e+03,3.683810e+03,3.696744e+03,3.719608e+03,3.767628e+03,3872.142,3992.188,4161.137


In [14]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [15]:
AccountsData.columns = ['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '2018']

In [16]:
AccountsData.columns

Index(['Country Name', 'Country Code', 'Series Name', 'Series Code', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018'],
      dtype='object')

In [17]:
AccountsData.drop(columns=['1996', '1997','1999', '2000','2002','2003','2005', '2006','2008', '2009','2011', '2012','2015', '2016'], inplace=True)

In [19]:
AccountsData.head()

,Country Name,Country Code,Series Name,Series Code,1995,1998,2001,2004,2007,2010,2013,2014,2017,2018
0,Albania,ALB,Human capital (constant 2018 US$),NW.HCA.TO,4.49E+10,3.88E+10,4.76E+10,5.23E+10,5.940000e+10,6.810000e+10,7.160000e+10,7.250000e+10,7.80E+10,8.12E+10
1,Albania,ALB,Human capital per capita (constant 2018 US$),NW.HCA.PC,14072.79,12399.48,15564.46,17262.12,1.999908e+04,2.336286e+04,2.473162e+04,2.509002e+04,27160.86,28333.68
2,Albania,ALB,"Human capital per capita, employed female (con...",NW.HCA.FEMP.PC,3468.858,3013.391,3783.821,2086.846,2.465247e+03,2.889150e+03,2.954273e+03,2.957420e+03,3178.417,3312.927
3,Albania,ALB,"Human capital per capita, employed male (const...",NW.HCA.MEMP.PC,6507.931,5901.072,7607.771,10969.67,1.308602e+04,1.578992e+04,1.681013e+04,1.699138e+04,18445.94,19245.19
4,Albania,ALB,"Human capital per capita, female (constant 201...",NW.HCA.FEMA.PC,4893.006,4191.445,5169.876,2759.035,3.170332e+03,3.613606e+03,3.696744e+03,3.719608e+03,3992.188,4161.137


In [20]:
AccountsSeries = pd.read_csv('Dataset/Wealth-AccountSeries.csv')

In [21]:
AccountsSeries.head()

,Code,Indicator Name,Long definition,Source,Topic,Unit of measure,Periodicity,Reference period,Statistical concept and methodology,Previous Indicator Code,Previous Indicator Name
0,NW.HCA.TO,Human capital (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
1,NW.HCA.PC,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
2,NW.HCA.FEMP.PC,"Human capital per capita, employed female (con...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
3,NW.HCA.MEMP.PC,"Human capital per capita, employed male (const...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN
4,NW.HCA.FEMA.PC,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...,World Bank. 2021. The Changing Wealth of Natio...,Human capital,Constant 2018 US$,Annual,1995-2018,Total wealth is calculated by summing up estim...,NaN,NaN


In [22]:
AccountsSeries.columns

Index(['Code', 'Indicator Name', 'Long definition', 'Source', 'Topic',
       'Unit of measure', 'Periodicity', 'Reference period',
       'Statistical concept and methodology', 'Previous Indicator Code',
       'Previous Indicator Name'],
      dtype='object')

In [23]:
AccountsSeries = AccountsSeries[['Code','Topic', 'Indicator Name', 'Long definition']]

In [24]:
AccountsSeries

,Code,Topic,Indicator Name,Long definition
0,NW.HCA.TO,Human capital,Human capital (constant 2018 US$),Human capital is computed as the present value...
1,NW.HCA.PC,Human capital,Human capital per capita (constant 2018 US$),Human capital is computed as the present value...
2,NW.HCA.FEMP.PC,Human capital,"Human capital per capita, employed female (con...",Human capital is computed as the present value...
3,NW.HCA.MEMP.PC,Human capital,"Human capital per capita, employed male (const...",Human capital is computed as the present value...
4,NW.HCA.FEMA.PC,Human capital,"Human capital per capita, female (constant 201...",Human capital is computed as the present value...
5,NW.HCA.MALE.PC,Human capital,"Human capital per capita, male (constant 2018 ...",Human capital is computed as the present value...
6,NW.HCA.FSEM.PC,Human capital,"Human capital per capita, self-employed female...",Human capital is computed as the present value...
7,NW.HCA.MSEM.PC,Human capital,"Human capital per capita, self-employed male (...",Human capital is computed as the present value...
8,NW.HCA.FEMP.TO,Human capital,"Human capital, employed female (constant 2018 ...",Human capital is computed as the present value...
9,NW.HCA.MEMP.TO,Human capital,"Human capital, employed male (constant 2018 US$)",Human capital is computed as the present value...


## Create Database "accounts" and Tables

In [25]:
cur, conn = create_database()

In [26]:
account_country_table = ("""CREATE TABLE IF NOT EXISTS accountscountry (
country_code VARCHAR PRIMARY KEY,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR)""")

In [27]:
cur.execute(account_country_table)
conn.commit()

In [28]:
account_data_table = ("""CREATE TABLE IF NOT EXISTS accountsdata (
country_name VARCHAR,
country_code VARCHAR,
series_name VARCHAR,
series_code VARCHAR,
year_1995 VARCHAR,year_1998 VARCHAR,year_2001 VARCHAR,year_2004 VARCHAR,year_2007 numeric,
year_2010 numeric,year_2013 numeric,year_2014 numeric,year_2017 VARCHAR,year_2018 VARCHAR)""")
cur.execute(account_data_table)
conn.commit()

In [29]:
account_series_table = ("""CREATE TABLE IF NOT EXISTS accountsseries (
series_code VARCHAR,
topic VARCHAR,
indicator_name VARCHAR,
long_definition VARCHAR)""")
cur.execute(account_series_table)
conn.commit()


### Inserting Data Into Tables

In [30]:
account_country_table_insert = ("""INSERT INTO accountscountry (
country_code,
short_name,
table_name,
long_name,
currency_unit) VALUES (%s,%s,%s,%s,%s)""")

In [31]:
for i, row in AccountsCountry_clean.iterrows():
    print(i,list(row))

0 ['ALB', 'Albania', 'Albania', 'Republic of Albania', 'Albanian lek']
1 ['ARG', 'Argentina', 'Argentina', 'Argentine Republic', 'Argentine peso']
2 ['ARM', 'Armenia', 'Armenia', 'Republic of Armenia', 'Armenian dram']
3 ['AUS', 'Australia', 'Australia', 'Commonwealth of Australia', 'Australian dollar']
4 ['AUT', 'Austria', 'Austria', 'Republic of Austria', 'Euro']
5 ['AZE', 'Azerbaijan', 'Azerbaijan', 'Republic of Azerbaijan', 'New Azeri manat']
6 ['BHR', 'Bahrain', 'Bahrain', 'Kingdom of Bahrain', 'Bahraini dinar']
7 ['BGD', 'Bangladesh', 'Bangladesh', "People's Republic of Bangladesh", 'Bangladeshi taka']
8 ['BLR', 'Belarus', 'Belarus', 'Republic of Belarus', 'Belarusian rubel']
9 ['BEL', 'Belgium', 'Belgium', 'Kingdom of Belgium', 'Euro']
10 ['BLZ', 'Belize', 'Belize', 'Belize', 'Belize dollar']
11 ['BEN', 'Benin', 'Benin', 'Republic of Benin', 'West African CFA franc']
12 ['BOL', 'Bolivia', 'Bolivia', 'Plurinational State of Bolivia', 'Bolivian Boliviano']
13 ['BIH', 'Bosnia and H

548 ['BLZ', nan, nan, 'Belize', nan]
549 ['BLZ', nan, nan, 'Belize', nan]
550 ['BLZ', nan, nan, 'Belize', nan]
551 ['BLZ', nan, nan, 'Belize', nan]
552 ['BLZ', nan, nan, 'Belize', nan]
553 ['BLZ', nan, nan, 'Belize', nan]
554 ['BLZ', nan, nan, 'Belize', nan]
555 ['BLZ', nan, nan, 'Belize', nan]
556 ['BLZ', nan, nan, 'Belize', nan]
557 ['BLZ', nan, nan, 'Belize', nan]
558 ['BLZ', nan, nan, 'Belize', nan]
559 ['BLZ', nan, nan, 'Belize', nan]
560 ['BLZ', nan, nan, 'Belize', nan]
561 ['BLZ', nan, nan, 'Belize', nan]
562 ['BLZ', nan, nan, 'Belize', nan]
563 ['BLZ', nan, nan, 'Belize', nan]
564 ['BLZ', nan, nan, 'Belize', nan]
565 ['BLZ', nan, nan, 'Belize', nan]
566 ['BLZ', nan, nan, 'Belize', nan]
567 ['BLZ', nan, nan, 'Belize', nan]
568 ['BLZ', nan, nan, 'Belize', nan]
569 ['BLZ', nan, nan, 'Belize', nan]
570 ['BLZ', nan, nan, 'Belize', nan]
571 ['BLZ', nan, nan, 'Belize', nan]
572 ['BEN', nan, nan, 'Benin', nan]
573 ['BEN', nan, nan, 'Benin', nan]
574 ['BEN', nan, nan, 'Benin', nan]
575 

1092 ['CAN', nan, nan, 'Canada', nan]
1093 ['CAN', nan, nan, 'Canada', nan]
1094 ['CAN', nan, nan, 'Canada', nan]
1095 ['CAN', nan, nan, 'Canada', nan]
1096 ['CAN', nan, nan, 'Canada', nan]
1097 ['CAN', nan, nan, 'Canada', nan]
1098 ['CAN', nan, nan, 'Canada', nan]
1099 ['CAN', nan, nan, 'Canada', nan]
1100 ['CAN', nan, nan, 'Canada', nan]
1101 ['CAN', nan, nan, 'Canada', nan]
1102 ['CAN', nan, nan, 'Canada', nan]
1103 ['CAN', nan, nan, 'Canada', nan]
1104 ['CAN', nan, nan, 'Canada', nan]
1105 ['CAN', nan, nan, 'Canada', nan]
1106 ['CAN', nan, nan, 'Canada', nan]
1107 ['CAN', nan, nan, 'Canada', nan]
1108 ['CAN', nan, nan, 'Canada', nan]
1109 ['CAN', nan, nan, 'Canada', nan]
1110 ['CAN', nan, nan, 'Canada', nan]
1111 ['CAN', nan, nan, 'Canada', nan]
1112 ['CAN', nan, nan, 'Canada', nan]
1113 ['CAN', nan, nan, 'Canada', nan]
1114 ['CAN', nan, nan, 'Canada', nan]
1115 ['CAN', nan, nan, 'Canada', nan]
1116 ['CAN', nan, nan, 'Canada', nan]
1117 ['CAN', nan, nan, 'Canada', nan]
1118 ['CAN',

1738 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1739 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1740 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1741 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1742 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1743 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1744 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1745 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1746 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1747 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1748 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1749 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1750 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1751 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1752 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1753 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1754 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1755 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1756 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1757 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1758 ['COD', nan, nan, 'Dem. Rep. Congo', nan]
1759 ['COD', 

2373 ['GAB', nan, nan, 'Gabon', nan]
2374 ['GAB', nan, nan, 'Gabon', nan]
2375 ['GAB', nan, nan, 'Gabon', nan]
2376 ['GAB', nan, nan, 'Gabon', nan]
2377 ['GAB', nan, nan, 'Gabon', nan]
2378 ['GAB', nan, nan, 'Gabon', nan]
2379 ['GAB', nan, nan, 'Gabon', nan]
2380 ['GAB', nan, nan, 'Gabon', nan]
2381 ['GAB', nan, nan, 'Gabon', nan]
2382 ['GAB', nan, nan, 'Gabon', nan]
2383 ['GAB', nan, nan, 'Gabon', nan]
2384 ['GAB', nan, nan, 'Gabon', nan]
2385 ['GAB', nan, nan, 'Gabon', nan]
2386 ['GAB', nan, nan, 'Gabon', nan]
2387 ['GAB', nan, nan, 'Gabon', nan]
2388 ['GAB', nan, nan, 'Gabon', nan]
2389 ['GAB', nan, nan, 'Gabon', nan]
2390 ['GAB', nan, nan, 'Gabon', nan]
2391 ['GAB', nan, nan, 'Gabon', nan]
2392 ['GEO', nan, nan, 'Georgia', nan]
2393 ['GEO', nan, nan, 'Georgia', nan]
2394 ['GEO', nan, nan, 'Georgia', nan]
2395 ['GEO', nan, nan, 'Georgia', nan]
2396 ['GEO', nan, nan, 'Georgia', nan]
2397 ['GEO', nan, nan, 'Georgia', nan]
2398 ['GEO', nan, nan, 'Georgia', nan]
2399 ['GEO', nan, nan, '

...

4753 ['NAM', nan, nan, 'Namibia', nan]
4754 ['NAM', nan, nan, 'Namibia', nan]
4755 ['NAM', nan, nan, 'Namibia', nan]
4756 ['NAM', nan, nan, 'Namibia', nan]
4757 ['NAM', nan, nan, 'Namibia', nan]
4758 ['NAM', nan, nan, 'Namibia', nan]
4759 ['NAM', nan, nan, 'Namibia', nan]
4760 ['NAM', nan, nan, 'Namibia', nan]
4761 ['NAM', nan, nan, 'Namibia', nan]
4762 ['NAM', nan, nan, 'Namibia', nan]
4763 ['NAM', nan, nan, 'Namibia', nan]
4764 ['NAM', nan, nan, 'Namibia', nan]
4765 ['NAM', nan, nan, 'Namibia', nan]
4766 ['NAM', nan, nan, 'Namibia', nan]
4767 ['NAM', nan, nan, 'Namibia', nan]
4768 ['NAM', nan, nan, 'Namibia', nan]
4769 ['NAM', nan, nan, 'Namibia', nan]
4770 ['NAM', nan, nan, 'Namibia', nan]
4771 ['NAM', nan, nan, 'Namibia', nan]
4772 ['NAM', nan, nan, 'Namibia', nan]
4773 ['NAM', nan, nan, 'Namibia', nan]
4774 ['NAM', nan, nan, 'Namibia', nan]
4775 ['NAM', nan, nan, 'Namibia', nan]
4776 ['NAM', nan, nan, 'Namibia', nan]
4777 ['NAM', nan, nan, 'Namibia', nan]
4778 ['NAM', nan, nan, 'N

5351 ['PNG', nan, nan, 'Papua New Guinea', nan]
5352 ['PNG', nan, nan, 'Papua New Guinea', nan]
5353 ['PNG', nan, nan, 'Papua New Guinea', nan]
5354 ['PNG', nan, nan, 'Papua New Guinea', nan]
5355 ['PNG', nan, nan, 'Papua New Guinea', nan]
5356 ['PRY', nan, nan, 'Paraguay', nan]
5357 ['PRY', nan, nan, 'Paraguay', nan]
5358 ['PRY', nan, nan, 'Paraguay', nan]
5359 ['PRY', nan, nan, 'Paraguay', nan]
5360 ['PRY', nan, nan, 'Paraguay', nan]
5361 ['PRY', nan, nan, 'Paraguay', nan]
5362 ['PRY', nan, nan, 'Paraguay', nan]
5363 ['PRY', nan, nan, 'Paraguay', nan]
5364 ['PRY', nan, nan, 'Paraguay', nan]
5365 ['PRY', nan, nan, 'Paraguay', nan]
5366 ['PRY', nan, nan, 'Paraguay', nan]
5367 ['PRY', nan, nan, 'Paraguay', nan]
5368 ['PRY', nan, nan, 'Paraguay', nan]
5369 ['PRY', nan, nan, 'Paraguay', nan]
5370 ['PRY', nan, nan, 'Paraguay', nan]
5371 ['PRY', nan, nan, 'Paraguay', nan]
5372 ['PRY', nan, nan, 'Paraguay', nan]
5373 ['PRY', nan, nan, 'Paraguay', nan]
5374 ['PRY', nan, nan, 'Paraguay', nan]


5947 ['SLE', nan, nan, 'Sierra Leone', nan]
5948 ['SLE', nan, nan, 'Sierra Leone', nan]
5949 ['SLE', nan, nan, 'Sierra Leone', nan]
5950 ['SLE', nan, nan, 'Sierra Leone', nan]
5951 ['SLE', nan, nan, 'Sierra Leone', nan]
5952 ['SLE', nan, nan, 'Sierra Leone', nan]
5953 ['SLE', nan, nan, 'Sierra Leone', nan]
5954 ['SLE', nan, nan, 'Sierra Leone', nan]
5955 ['SLE', nan, nan, 'Sierra Leone', nan]
5956 ['SLE', nan, nan, 'Sierra Leone', nan]
5957 ['SLE', nan, nan, 'Sierra Leone', nan]
5958 ['SLE', nan, nan, 'Sierra Leone', nan]
5959 ['SLE', nan, nan, 'Sierra Leone', nan]
5960 ['SLE', nan, nan, 'Sierra Leone', nan]
5961 ['SLE', nan, nan, 'Sierra Leone', nan]
5962 ['SLE', nan, nan, 'Sierra Leone', nan]
5963 ['SLE', nan, nan, 'Sierra Leone', nan]
5964 ['SLE', nan, nan, 'Sierra Leone', nan]
5965 ['SLE', nan, nan, 'Sierra Leone', nan]
5966 ['SLE', nan, nan, 'Sierra Leone', nan]
5967 ['SLE', nan, nan, 'Sierra Leone', nan]
5968 ['SLE', nan, nan, 'Sierra Leone', nan]
5969 ['SLE', nan, nan, 'Sierra L

6521 ['TJK', nan, nan, 'Tajikistan', nan]
6522 ['TJK', nan, nan, 'Tajikistan', nan]
6523 ['TJK', nan, nan, 'Tajikistan', nan]
6524 ['TJK', nan, nan, 'Tajikistan', nan]
6525 ['TJK', nan, nan, 'Tajikistan', nan]
6526 ['TJK', nan, nan, 'Tajikistan', nan]
6527 ['TJK', nan, nan, 'Tajikistan', nan]
6528 ['TJK', nan, nan, 'Tajikistan', nan]
6529 ['TJK', nan, nan, 'Tajikistan', nan]
6530 ['TJK', nan, nan, 'Tajikistan', nan]
6531 ['TJK', nan, nan, 'Tajikistan', nan]
6532 ['TJK', nan, nan, 'Tajikistan', nan]
6533 ['TJK', nan, nan, 'Tajikistan', nan]
6534 ['TJK', nan, nan, 'Tajikistan', nan]
6535 ['TJK', nan, nan, 'Tajikistan', nan]
6536 ['TJK', nan, nan, 'Tajikistan', nan]
6537 ['TJK', nan, nan, 'Tajikistan', nan]
6538 ['TJK', nan, nan, 'Tajikistan', nan]
6539 ['TJK', nan, nan, 'Tajikistan', nan]
6540 ['TJK', nan, nan, 'Tajikistan', nan]
6541 ['TJK', nan, nan, 'Tajikistan', nan]
6542 ['TJK', nan, nan, 'Tajikistan', nan]
6543 ['TJK', nan, nan, 'Tajikistan', nan]
6544 ['TJK', nan, nan, 'Tajikistan

7125 ['GBR', nan, nan, 'United Kingdom', nan]
7126 ['GBR', nan, nan, 'United Kingdom', nan]
7127 ['GBR', nan, nan, 'United Kingdom', nan]
7128 ['GBR', nan, nan, 'United Kingdom', nan]
7129 ['GBR', nan, nan, 'United Kingdom', nan]
7130 ['GBR', nan, nan, 'United Kingdom', nan]
7131 ['GBR', nan, nan, 'United Kingdom', nan]
7132 ['GBR', nan, nan, 'United Kingdom', nan]
7133 ['GBR', nan, nan, 'United Kingdom', nan]
7134 ['GBR', nan, nan, 'United Kingdom', nan]
7135 ['GBR', nan, nan, 'United Kingdom', nan]
7136 ['GBR', nan, nan, 'United Kingdom', nan]
7137 ['GBR', nan, nan, 'United Kingdom', nan]
7138 ['GBR', nan, nan, 'United Kingdom', nan]
7139 ['GBR', nan, nan, 'United Kingdom', nan]
7140 ['GBR', nan, nan, 'United Kingdom', nan]
7141 ['GBR', nan, nan, 'United Kingdom', nan]
7142 ['GBR', nan, nan, 'United Kingdom', nan]
7143 ['GBR', nan, nan, 'United Kingdom', nan]
7144 ['GBR', nan, nan, 'United Kingdom', nan]
7145 ['GBR', nan, nan, 'United Kingdom', nan]
7146 ['GBR', nan, nan, 'United Kin

In [32]:
AccountsCountry_clean=AccountsCountry_clean.head(146)

In [33]:
for i, row in AccountsCountry_clean.iterrows():
        cur.execute(account_country_table_insert, list(row))

In [34]:
conn.commit()

In [35]:
AccountsData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7592 entries, 0 to 7591
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  7592 non-null   object 
 1   Country Code  7592 non-null   object 
 2   Series Name   7592 non-null   object 
 3   Series Code   7592 non-null   object 
 4   1995          7592 non-null   object 
 5   1998          7592 non-null   object 
 6   2001          7592 non-null   object 
 7   2004          7592 non-null   object 
 8   2007          7592 non-null   float64
 9   2010          7592 non-null   float64
 10  2013          7592 non-null   float64
 11  2014          7592 non-null   float64
 12  2017          7592 non-null   object 
 13  2018          7592 non-null   object 
dtypes: float64(4), object(10)
memory usage: 830.5+ KB


In [36]:
account_data_table_insert = ("""INSERT INTO accountsdata (
country_name,
country_code,
series_name,
series_code,
year_1995,year_1998,year_2001,year_2004,year_2007,year_2010,year_2013,year_2014,year_2017,year_2018)
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)""")

In [37]:
for i, row in AccountsData.iterrows():
        cur.execute(account_data_table_insert, list(row))

In [38]:
for i, row in AccountsData.iterrows():
        print(list(row))

['Albania', 'ALB', 'Human capital (constant 2018 US$)', 'NW.HCA.TO', '4.49E+10', '3.88E+10', '4.76E+10', '5.23E+10', 59400000000.0, 68100000000.0, 71600000000.0, 72500000000.0, '7.80E+10', '8.12E+10']
['Albania', 'ALB', 'Human capital per capita (constant 2018 US$)', 'NW.HCA.PC', '14072.79', '12399.48', '15564.46', '17262.12', 19999.08, 23362.86, 24731.62, 25090.02, '27160.86', '28333.68']
['Albania', 'ALB', 'Human capital per capita, employed female (constant 2018 US$)', 'NW.HCA.FEMP.PC', '3468.858', '3013.391', '3783.821', '2086.846', 2465.247, 2889.15, 2954.273, 2957.42, '3178.417', '3312.927']
['Albania', 'ALB', 'Human capital per capita, employed male (constant 2018 US$)', 'NW.HCA.MEMP.PC', '6507.931', '5901.072', '7607.771', '10969.67', 13086.02, 15789.92, 16810.13, 16991.38, '18445.94', '19245.19']
['Albania', 'ALB', 'Human capital per capita, female (constant 2018 US$)', 'NW.HCA.FEMA.PC', '4893.006', '4191.445', '5169.876', '2759.035', 3170.332, 3613.606, 3696.744, 3719.608, '3

['Bulgaria', 'BGR', 'Natural capital per capita (constant 2018 US$)', 'NW.NCA.PC', '5880.584', '6277.195', '6316.972', '6126.413', 7021.912, 7753.37, 8751.021, 9348.031, '7664.072', '7446.758']
['Bulgaria', 'BGR', 'Natural capital per capita, agricultural land (constant 2018 US$)', 'NW.NCA.AGRI.PC', '4015.1', '4412.021', '4280.688', '3369.931', 3181.442, 2751.663, 3115.376, 3368.675, '3445.948', '3330.269']
['Bulgaria', 'BGR', 'Natural capital per capita, agricultural land: cropland (constant 2018 US$)', 'NW.NCA.CROL.PC', '2500.538', '2965.998', '2817.45', '2189.428', 2268.002, 1935.039, 2309.48, 2534.873, '2685.854', '2609.133']
['Bulgaria', 'BGR', 'Natural capital per capita, agricultural land: pastureland (constant 2018 US$)', 'NW.NCA.PASL.PC', '1514.562', '1446.022', '1463.237', '1180.503', 913.4399, 816.6247, 805.8956, 833.802, '760.0942', '721.1365']
['Bulgaria', 'BGR', 'Natural capital per capita, fisheries (constant 2018 US$)', 'NW.NCA.FISH.PC', '29.33552', '35.12714', '24.5315

['Congo', 'COG', 'Natural capital, nonrenewable assets: minerals (constant 2018 US$)', 'NW.NCA.MINR.TO', '2.25E+08', '40229509', '3550350', '1363826', 4844749.0, 10563121.0, 18878347.0, 19782839.0, '18172954', '15640314']
['Congo', 'COG', 'Natural capital, nonrenewable assets: oil (constant 2018 US$)', 'NW.NCA.SAOI.TO', '1.44E+10', '2.08E+10', '2.71E+10', '3.36E+10', 46200000000.0, 43900000000.0, 60200000000.0, 60600000000.0, '3.91E+10', '3.60E+10']
['Congo', 'COG', 'Natural capital, protected areas (constant 2018 US$)', 'NW.NCA.PRAR.TO', '1.39E+08', '1.53E+08', '3.72E+08', '8.18E+08', 1120000000.0, 1490000000.0, 2200000000.0, 2310000000.0, '3.25E+09', '3.24E+09']
['Congo', 'COG', 'Natural capital, renewable (constant 2018 US$)', 'NW.NCA.RNEW.TO', '2.52E+10', '2.65E+10', '2.52E+10', '2.56E+10', 26900000000.0, 28400000000.0, 30200000000.0, 30200000000.0, '3.68E+10', '3.72E+10']
['Congo', 'COG', 'Net foreign assets (constant 2018 US$)', 'NW.NFA.TO', '-1.70E+10', '-1.70E+10', '-1.10E+10',

['Germany', 'DEU', 'Human capital per capita, self-employed female (constant 2018 US$)', 'NW.HCA.FSEM.PC', '10822.78', '9946.887', '9300.294', '8471.815', 9170.062, 9710.166, 11142.77, 11024.28, '10555.43', '10644.11']
['Germany', 'DEU', 'Human capital per capita, self-employed male (constant 2018 US$)', 'NW.HCA.MSEM.PC', '21652.76', '19968.56', '18656.5', '16932.06', 19482.45, 19952.52, 23752.59, 23555.31, '22752.28', '23029.1']
['Germany', 'DEU', 'Human capital, employed female (constant 2018 US$)', 'NW.HCA.FEMP.TO', '6.47E+12', '6.41E+12', '6.76E+12', '6.57E+12', 7150000000000.0, 7670000000000.0, 8410000000000.0, 8540000000000.0, '9.02E+12', '9.12E+12']
['Germany', 'DEU', 'Human capital, employed male (constant 2018 US$)', 'NW.HCA.MEMP.TO', '1.29E+13', '1.29E+13', '1.36E+13', '1.31E+13', 15200000000000.0, 15800000000000.0, 17900000000000.0, 18200000000000.0, '1.94E+13', '1.97E+13']
['Germany', 'DEU', 'Human capital, female (constant 2018 US$)', 'NW.HCA.FEMA.TO', '7.35E+12', '7.23E+1

['Ireland', 'IRL', 'Natural capital, agricultural land: cropland (constant 2018 US$)', 'NW.NCA.CROL.TO', '5.00E+09', '4.58E+09', '3.96E+09', '3.50E+09', 3830000000.0, 3750000000.0, 4050000000.0, 4110000000.0, '3.80E+09', '3.80E+09']
['Ireland', 'IRL', 'Natural capital, agricultural land: pastureland (constant 2018 US$)', 'NW.NCA.PASL.TO', '5.85E+10', '5.25E+10', '4.56E+10', '3.97E+10', 33500000000.0, 33400000000.0, 39400000000.0, 42500000000.0, '4.28E+10', '4.27E+10']
['Ireland', 'IRL', 'Natural capital, fisheries (constant 2018 US$)', 'NW.NCA.FISH.TO', '8.97E+08', '9.46E+08', '3.05E+08', '5.68E+08', 3310000000.0, 4230000000.0, 6220000000.0, 5730000000.0, '6.68E+08', '6.00E+08']
['Ireland', 'IRL', 'Natural capital, forests: ecosystem services (constant 2018 US$)', 'NW.NCA.FECO.TO', '5.23E+09', '6.52E+09', '7.56E+09', '9.91E+09', 13500000000.0, 10100000000.0, 10400000000.0, 13300000000.0, '1.97E+10', '1.95E+10']
['Ireland', 'IRL', 'Natural capital, forests: timber (constant 2018 US$)', 

['Madagascar', 'MDG', 'Natural capital, agricultural land: pastureland (constant 2018 US$)', 'NW.NCA.PASL.TO', '1.18E+10', '9.44E+09', '9.77E+09', '8.88E+09', 9590000000.0, 13300000000.0, 14400000000.0, 14900000000.0, '1.62E+10', '1.56E+10']
['Madagascar', 'MDG', 'Natural capital, fisheries (constant 2018 US$)', 'NW.NCA.FISH.TO', '8.25E+08', '7.54E+08', '8.24E+08', '1.10E+09', 556000000.0, 415000000.0, 742000000.0, 221000000.0, '4.68E+08', '4.16E+08']
['Madagascar', 'MDG', 'Natural capital, forests: ecosystem services (constant 2018 US$)', 'NW.NCA.FECO.TO', '8.67E+09', '8.63E+09', '8.43E+09', '8.72E+09', 8750000000.0, 8810000000.0, 8980000000.0, 9110000000.0, '9.29E+09', '9.34E+09']
['Madagascar', 'MDG', 'Natural capital, forests: timber (constant 2018 US$)', 'NW.NCA.FTIM.TO', '9.71E+09', '1.08E+10', '8.19E+09', '8.74E+09', 13300000000.0, 15800000000.0, 16600000000.0, 17100000000.0, '2.31E+10', '2.25E+10']
['Madagascar', 'MDG', 'Natural capital, fossil fuels (constant 2018 US$)', 'NW.N

['Nepal', 'NPL', 'Natural capital, fisheries (constant 2018 US$)', 'NW.NCA.FISH.TO', '0', '0', '0', '0', 0.0, 0.0, 0.0, 0.0, '0', '0']
['Nepal', 'NPL', 'Natural capital, forests: ecosystem services (constant 2018 US$)', 'NW.NCA.FECO.TO', '4.99E+09', '4.70E+09', '4.37E+09', '4.27E+09', 4140000000.0, 4220000000.0, 4240000000.0, 4260000000.0, '4.20E+09', '4.16E+09']
['Nepal', 'NPL', 'Natural capital, forests: timber (constant 2018 US$)', 'NW.NCA.FTIM.TO', '3.60E+09', '3.35E+09', '3.54E+09', '3.56E+09', 3840000000.0, 4890000000.0, 5380000000.0, 5570000000.0, '5.74E+09', '5.36E+09']
['Nepal', 'NPL', 'Natural capital, fossil fuels (constant 2018 US$)', 'NW.NCA.FOSL.TO', '0', '0', '4281240', '11271085', 19661067.0, 31058522.0, 26357549.0, 26026120.0, '22990304', '23015562']
['Nepal', 'NPL', 'Natural capital, mangroves (constant 2018 US$)', 'NW.NCA.MANG.TO', '0', '0', '0', '0', 0.0, 0.0, 0.0, 0.0, '0', '0']
['Nepal', 'NPL', 'Natural capital, nonrenewable assets (constant 2018 US$)', 'NW.NCA.SS

['Philippines', 'PHL', 'Natural capital per capita, forests: timber (constant 2018 US$)', 'NW.NCA.FTIM.PC', '187.8603', '180.7084', '127.2595', '127.0753', 141.0833, 170.8625, 174.7743, 179.6321, '187.9375', '182.4495']
['Philippines', 'PHL', 'Natural capital per capita, fossil fuels (constant 2018 US$)', 'NW.NCA.FOSL.PC', '16.23033', '6.246489', '6.031762', '53.19826', 100.2505, 116.2961, 134.2713, 121.971, '91.93808', '99.17337']
['Philippines', 'PHL', 'Natural capital per capita, mangroves (constant 2018 US$)', 'NW.NCA.MANG.PC', '161.183', '150.2705', '140.4994', '131.8252', 124.6006, 118.2648, 71.88721, 57.43008, '16.80191', '4.050468']
['Philippines', 'PHL', 'Natural capital per capita, nonrenewable assets (constant 2018 US$)', 'NW.NCA.SSOI.PC', '65.14432', '41.83043', '39.34418', '82.5727', 200.26, 292.4581, 345.7531, 317.4068, '249.6016', '237.5015']
['Philippines', 'PHL', 'Natural capital per capita, nonrenewable assets: coal (constant 2018 US$)', 'NW.NCA.SACO.PC', '1.428949', 

['Sri Lanka', 'LKA', 'Natural capital per capita, nonrenewable assets: coal (constant 2018 US$)', 'NW.NCA.SACO.PC', '0', '0', '0', '0', 0.0, 0.0, 0.0, 0.0, '0', '0']
['Sri Lanka', 'LKA', 'Natural capital per capita, nonrenewable assets: gas (constant 2018 US$)', 'NW.NCA.SAGA.PC', '0', '0', '0', '0', 0.0, 0.0, 0.0, 0.0, '0', '0']
['Sri Lanka', 'LKA', 'Natural capital per capita, nonrenewable assets: minerals (constant 2018 US$)', 'NW.NCA.MINR.PC', '0', '0', '0', '0', 0.445742, 5.996738, 6.981189, 5.493409, '3.160814', '2.632519']
['Sri Lanka', 'LKA', 'Natural capital per capita, nonrenewable assets: oil (constant 2018 US$)', 'NW.NCA.SAOI.PC', '0', '0', '0', '0', 0.0, 0.0, 0.0, 0.0, '0', '0']
['Sri Lanka', 'LKA', 'Natural capital per capita, protected areas (constant 2018 US$)', 'NW.NCA.PRAR.PC', '396.4518', '373.7214', '282.7527', '234.7126', 223.9068, 218.8561, 250.5185, 252.6929, '485.0869', '554.268']
['Sri Lanka', 'LKA', 'Natural capital per capita, renewable (constant 2018 US$)', '

['Uruguay', 'URY', 'Human capital (constant 2018 US$)', 'NW.HCA.TO', '2.38E+11', '2.99E+11', '2.96E+11', '2.26E+11', 298000000000.0, 409000000000.0, 437000000000.0, 451000000000.0, '4.58E+11', '4.64E+11']
['Uruguay', 'URY', 'Human capital per capita (constant 2018 US$)', 'NW.HCA.PC', '73893.69', '90940.37', '89133.41', '68180.59', 89579.96, 121740.2, 128938.0, 132564.3, '133303.3', '134638']
['Uruguay', 'URY', 'Human capital per capita, employed female (constant 2018 US$)', 'NW.HCA.FEMP.PC', '25924.85', '32653.49', '32797.18', '27350.55', 38371.47, 51959.21, 54076.08, 55518.0, '55358.86', '55860.32']
['Uruguay', 'URY', 'Human capital per capita, employed male (constant 2018 US$)', 'NW.HCA.MEMP.PC', '35547.37', '42188.24', '40556.59', '30195.84', 36500.41, 49792.51, 53691.64, 55280.67, '56057.42', '56671.51']
['Uruguay', 'URY', 'Human capital per capita, female (constant 2018 US$)', 'NW.HCA.FEMA.PC', '31163.39', '39677.34', '39852.41', '32404.75', 45909.29, 62166.25, 64698.97, 66424.14,

In [39]:
account_series_table_insert = ("""INSERT INTO accountsseries (
series_code,
topic,
indicator_name,
long_definition) VALUES (%s,%s,%s,%s)""")

In [40]:
for i, row in AccountsSeries.iterrows():
        cur.execute(account_series_table_insert, list(row))

In [41]:
conn.commit()

## Validate you data was inserted into table

In [48]:
cur.execute("SELECT * FROM accountscountry;")

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('ALB', 'Albania', 'Albania', 'Republic of Albania', 'Albanian lek')
('ARG', 'Argentina', 'Argentina', 'Argentine Republic', 'Argentine peso')
('ARM', 'Armenia', 'Armenia', 'Republic of Armenia', 'Armenian dram')
('AUS', 'Australia', 'Australia', 'Commonwealth of Australia', 'Australian dollar')
('AUT', 'Austria', 'Austria', 'Republic of Austria', 'Euro')
('AZE', 'Azerbaijan', 'Azerbaijan', 'Republic of Azerbaijan', 'New Azeri manat')
('BHR', 'Bahrain', 'Bahrain', 'Kingdom of Bahrain', 'Bahraini dinar')
('BGD', 'Bangladesh', 'Bangladesh', "People's Republic of Bangladesh", 'Bangladeshi taka')
('BLR', 'Belarus', 'Belarus', 'Republic of Belarus', 'Belarusian rubel')
('BEL', 'Belgium', 'Belgium', 'Kingdom of Belgium', 'Euro')
('BLZ', 'Belize', 'Belize', 'Belize', 'Belize dollar')
('BEN', 'Benin', 'Benin', 'Republic of Benin', 'West African CFA franc')
('BOL', 'Bolivia', 'Bolivia', 'Plurinational State of Bolivia', 'Bolivian Boliviano')
('BIH', 'Bosnia and Herzegovina', 'Bosnia and Herzego

## And Finally Close your cursor and connection

In [50]:

cur.close()
conn.close()